In [1]:
from multiprocessing.pool import ThreadPool
from multiprocessing import cpu_count
from glob import glob
import os


def get_frame(checkpoint_dir, input_image_path, dir_of_frames, cpu_count=1): # use only 1 CPU to generate 1 frame
    output_image_path = os.path.join(dir_of_frames, "{}{}".format(checkpoint_dir.split('_')[-1], os.path.splitext(input_image_path)[-1]))    
    os.system("python evaluate_upgraded.py --checkpoint {} --in-path {} --out-path {} --device /cpu:{}".format(
        checkpoint_dir, input_image_path, output_image_path, cpu_count))

def get_frame_wrapper(args):
    get_frame(*args)

def create_progress_frames(parent_dir_of_checkpoints, input_image_path, output_dir_of_frames):
    checkpoint_dirs = (path for path in glob(os.path.join(parent_dir_of_checkpoints, '*')) 
                       if os.path.isdir(path) and 'checkpoint' in path)
    checkpoint_dirs = sorted(checkpoint_dirs, key=lambda path: int(path.split('_')[-1]))
    
    args_list = zip(checkpoint_dirs, [input_image_path] * len(checkpoint_dirs), [output_dir_of_frames] * len(checkpoint_dirs))
    pool = ThreadPool(cpu_count())
    temp = pool.map(get_frame_wrapper, args_list)
    pool.close()
    del pool

In [2]:
%%time
create_progress_frames('test_dir_small_data', 'examples/style/udnie.jpg', 'saved_frames')

#parent_dir_of_checkpoints = 'test_dir_small_data'
#input_image_path = 'examples/style/udnie.jpg'
#dir_of_frames = 'saved_frames'

CPU times: user 96 ms, sys: 84 ms, total: 180 ms
Wall time: 2min 7s
